<a href="https://colab.research.google.com/github/NikitaSUAI/EmotionRecognition/blob/main/Audio_feats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INIT

In [1]:
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

# !cp -r '/content/drive/MyDrive/OMG_EMO' 'OMG_EMO'

BASE_PATH = Path("/content/drive/MyDrive/OMG_EMO")

TRAIN_PATH = BASE_PATH / "train_set.csv"
VAL_PATH = BASE_PATH / "val_set.csv"
TEST_PATH = BASE_PATH / "test_set.csv"

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd

train_df = pd.read_csv(TRAIN_PATH).dropna()
val_df = pd.read_csv(VAL_PATH).dropna()
test_df = pd.read_csv(TEST_PATH).dropna()

# Culc feats

In [3]:
!pip install opensmile

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.5 MB 4.1 MB/s 
     |████████████████████████████████| 635 kB 22.7 MB/s 
     |████████████████████████████████| 48 kB 5.7 MB/s 
     |████████████████████████████████| 167 kB 69.6 MB/s 
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=169061 sha256=195a5d881d4506a35a2f6c10c84e0bbe2cdbb0019e6232f86aec92877fab3409
  Stored in directory: /root/.cache/pip/wheels/47/60/19/6d020fc92138ed1b113a18271e83ea4b5525fe770cb45b9a2e
Successfully built iso-639


In [47]:
import torchaudio
import torch
import torchaudio.functional as F
import opensmile
import pandas as pd
from tqdm import tqdm
import pickle

tqdm.pandas()

smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    feature_level=opensmile.FeatureLevel.Functionals,
)

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad',
                              model='silero_vad',
                              force_reload=True)
(get_speech_timestamps,
 _, read_audio,
 *_) = utils

Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


## Fbanks

In [48]:
def get_fbanks_features(wav_path):
  try:
    wav_path = BASE_PATH / "data" / wav_path
    wav, sr = torchaudio.load(wav_path)

    re_wav = F.resample(wav[0], sr, 16000)
    speech_timestamps = get_speech_timestamps(re_wav, model, sampling_rate=16000)

    hop_length = int(10 * 16)
    fnank_extractor = torchaudio.transforms.MelSpectrogram(hop_length=hop_length)

    fbanks = fnank_extractor(torch.Tensor(re_wav[np.newaxis, :])).numpy()[0]
    try:
      fbanks_vad = []
      for segment in speech_timestamps:
        feats = fbanks[:, int(segment['start'] / 160): int(segment['end'] / 160)]               
        fbanks_vad.append(feats)
      fbanks_vad = np.concatenate(fbanks_vad, axis=1)
      return fbanks, fbanks_vad
    except :
      return fbanks, fbanks
  except BaseException:
    return None, None 
a, b = get_fbanks_features(train_df.iloc[0].path)
a.shape, b.shape

/usr/local/lib/python3.7/dist-packages/torchaudio/functional/functional.py:540: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  "At least one mel filterbank has all zero values. "


((128, 575), (128, 411))

In [49]:
subset = train_df
segment_features = subset.path.progress_apply(get_fbanks_features)
segment_feats, full_feats = dict(), dict()
for (f1, f2), (_, row) in zip(segment_features, subset.iterrows()):
  segment_feats[row.path] = f1
  full_feats[row.path] = f2

with open("/content/drive/MyDrive/OMG_EMO/feats/fbanks_segment_train.pkl", "wb") as f:
  pickle.dump(segment_feats, f)

with open("/content/drive/MyDrive/OMG_EMO/feats/fbanks_full_train.pkl", "wb") as f:
  pickle.dump(full_feats, f)

100%|██████████| 1660/1660 [32:34<00:00,  1.18s/it]


In [50]:
subset = train_df
segment_features = subset.path.progress_apply(get_fbanks_features)

segment_feats, full_feats = dict(), dict()
for (f1, f2), (_, row) in zip(segment_features, subset.iterrows()):
  segment_feats[row.path] = f1
  full_feats[row.path] = f2

with open("/content/drive/MyDrive/OMG_EMO/feats/fbanks_segment_test.pkl", "wb") as f:
  pickle.dump(segment_feats, f)

with open("/content/drive/MyDrive/OMG_EMO/feats/fbanks_full_test.pkl", "wb") as f:
  pickle.dump(full_feats, f)


100%|██████████| 1660/1660 [05:52<00:00,  4.71it/s]


## OpenSmile Feats

In this section we extract eGeMAPSv02 set from opensmile with and without vad

In [42]:
def get_smile_features(wav_path):
  try:
    wav_path = BASE_PATH / "data" / wav_path
    wav, sr = torchaudio.load(wav_path)

    re_wav = F.resample(wav[0], sr, 16000)
    speech_timestamps = get_speech_timestamps(re_wav, model, sampling_rate=16000)

    smile_feats_full = smile.process_signal(
        wav[0].numpy(),
        sr
    )
    try:
      smile_feats_by_vocal_segments = []
      for segment in speech_timestamps:
        feats = smile.process_signal(
            wav[0, int(segment['start'] / 16000 * sr) : int(segment['end'] / 16000 * sr)].numpy(),
            sr
        )
        smile_feats_by_vocal_segments.append(feats)
      smile_feats_by_vocal_segments = pd.concat(smile_feats_by_vocal_segments)
      return smile_feats_by_vocal_segments, smile_feats_full
    except :
      return smile_feats_full, smile_feats_full
  except BaseException:
    return None, None 
# get_smile_features(train_df.iloc[0].path)

In [1]:
subset = train_df
segment_features = subset.path.progress_apply(get_smile_features)
segment_feats, full_feats = dict(), dict()
for (f1, f2), (_, row) in zip(segment_features, subset.iterrows()):
  segment_feats[row.path] = f1
  full_feats[row.path] = f2

with open("/content/drive/MyDrive/OMG_EMO/feats/smile_segment_train.pkl", "wb") as f:
  pickle.dump(segment_feats, f)

with open("/content/drive/MyDrive/OMG_EMO/feats/smile_full_train.pkl", "wb") as f:
  pickle.dump(full_feats, f)
  

NameError: ignored

In [ ]:
subset = train_df
segment_features = subset.path.progress_apply(get_smile_features)

segment_feats, full_feats = dict(), dict()
for (f1, f2), (_, row) in zip(segment_features, subset.iterrows()):
  segment_feats[row.path] = f1
  full_feats[row.path] = f2

with open("/content/drive/MyDrive/OMG_EMO/feats/smile_segment_test.pkl", "wb") as f:
  pickle.dump(segment_feats, f)

with open("/content/drive/MyDrive/OMG_EMO/feats/smile_full_test.pkl", "wb") as f:
  pickle.dump(full_feats, f)


100%|██████████| 337/337 [05:24<00:00,  1.04it/s]
